# ELEC-E7851 Computational User Interface Design

# Bandits (07.12.2018)
### Kashyap Todi (www.kashyaptodi.com)

In [1]:
import time
import random
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

## Defining a Bandit

We start by defining a Bandit.

A bandit has $n$ arms.<br/>
Each arm has a reward probability $\theta$.<br/>
When we pull an arm $i$, the bandit returns a reward with corresponding probability $i_\theta$.

Let's create a Bandit instance with 3 arms

## Defining a Generic Solver

A generic Bandit solver consists of:
1. bandit: A Bandit object
2. counts: Number of times each arm has been pulled
3. actions: A History of actions taken (arms pulled)
4. regret: Cumulative regret until current time step
5. regrets: A history of cumulative regrets

A solver must specify the steps taken during a single time step (`run_one_step`)

A solver can be run $num\ steps$ times, to calculate the cumulative regret and optionally the estimated $\theta$s.



## Define an Experiment
Let us now define an experiment to run the solver for n trials with m iterations each trial.

## Implement Solvers

### 1. Explore-Only:
First, let's write a random solver, that randomly explores at every iteration:

### 2. Exploit-Only:
Next, let's write a Greedy solver, that exploits a pre-determined arm at every iteration:

 ### 3. $\epsilon$-Greedy:
 
We can explore random arms with a fixed probability ($\epsilon$), while exploiting the best found arm otherwise.
 
After each time step, we obtain reward r for pulling arm i.<br/>
We update the estimated theta for arm i based on the outcome.

 ### 4. UCB1:
 
The Upper Confident Bound solver prefers arms with stronger potential for optimal value.

At each step, we pick the best arm $i$ to maximise the upper confidence bound

$a_t^{UCB}= argmax_{a\in A} \hat{Q_t}(a) + \hat{U_t}(a)$<br><br>
$\hat{U_t}(a) = \sqrt{\dfrac{2 \log t}{2N_t(a)}}$

 ### 5. Thompson Sampling:
 
The Thompson Sampling solver implements probability matching.

For Bernoulli bandit, Q(a) follows a Beta distribution

By default, $\alpha$ and $\beta$ are set to 1 (50% reward probability).<br>
We can set initial $\alpha$ and $\beta$ based on our prior knowledge of reward probability.<br>

After pulling an arm i, the $\alpha$ and $\beta$ for i is updated.


## Test the solvers

We now test the solvers by running the experiment and printing the mean cumulative regret and standard deviation.

## In-class task 1: 
- Modify the bandit instance (add more arms, change $\theta$s.
- Change the target arm for Greedy.
- Change $\epsilon$ for $\epsilon$-Greedy.
- Specify strong prior for Thompson Sampling ( $\alpha$ and $\beta$ ).

Observe and discuss how the results vary.

## In-class task 2:
- For one trial, plot a comparative graph showing cumulative regrets over time for each solver.
- Calculate *cumulative reward* over time for each solver, and construct the plot again.

Discuss how they compare.